In [1]:
import pandas as pd
from api import get_api
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium

In [2]:
# Cluster vrp 정보 가져오기
solution = pd.read_csv('solution_cluster_info.csv')

In [3]:
# 지점 정보 가져오기
spot = pd.read_csv('clusterinfo.csv')

In [4]:
def find(cNum):
    cNum = cNum[2:]
    result = pd.DataFrame()
    for i in cNum:
        if(i==50):
            break
        temp = spot[spot['cluster']==i]
        result = pd.concat([result, temp],ignore_index=True,axis=0)
        result = result[['lat','lng']]
    return result

In [5]:
solutionList = solution.values.tolist()
arrive = {'lat': 35.229546, 'lng' : 126.847516}

In [6]:
def calcdistance(A,B):
    return abs(A['lat']-B['lat']) + abs(A['lng']-B['lng'])

In [7]:
def calc_distance(originX,originY,destinationX, destinationY):
    # origin and destination should be coordinate
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {'origin': str(originX) +','+str(originY),
              'destination': str(destinationX) + ','+str(destinationY)}
    header = {'Authorization': 'KakaoAK ' + get_api()}

    route = requests.get(url, params=params, headers=header).json()
#     print(route)

    if route['routes'][0]['result_code'] == 104:
        return 0
    else:
        return(route['routes'][0]['summary']['duration'])

In [8]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/60)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance/60))

In [9]:
print('start')

start


In [10]:
answer = []


for i in solutionList:
    newDF = pd.DataFrame(find(i))
    if(len(newDF) <= 0):
        continue
    newDF.append(arrive,ignore_index=True)
    result = []
    for j in range(len(newDF)-1):
        temp = []
        for k in range(len(newDF)):
            distance = calcdistance(newDF.iloc[j],newDF.iloc[k]) * 10000
            temp.append(distance)
        result.append(temp)
    result.append([0]*len(newDF))
    
    def create_data_model():
        data = {}
        data['distance_matrix'] = result
        data['num_vehicles'] = 3
        data['depot'] = len(result)-1
        return data
    
    # Instantiate the data problem.

    data = create_data_model()
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)
    
    
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node] 
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        100000,  # no slack
        30000000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    def print_solution_only_num():
        output = []
        for vehicle_id in range(data['num_vehicles']):
            Output = []
            index = routing.Start(vehicle_id)
            while not routing.IsEnd(index):
                Output.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            Output.append(manager.IndexToNode(index))  
            output.append(Output)
        return output
    
    if solution:
        print_solution(data, manager, routing, solution)
        sol = print_solution_only_num()
        answer.append(sol)
        print(sol)
    else:
        print('No solution found !')

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2553/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


Objective: 0
Route for vehicle 0:
 217 -> 217
Distance of the route: 0.0m

Route for vehicle 1:
 217 -> 217
Distance of the route: 0.0m

Route for vehicle 2:
 217 ->  216 ->  215 ->  214 ->  213 ->  212 ->  211 ->  210 ->  209 ->  208 ->  207 ->  206 ->  205 ->  204 ->  203 ->  202 ->  201 ->  200 ->  199 ->  198 ->  197 ->  196 ->  195 ->  194 ->  193 ->  192 ->  191 ->  190 ->  189 ->  188 ->  187 ->  186 ->  185 ->  184 ->  183 ->  182 ->  181 ->  180 ->  179 ->  178 ->  177 ->  176 ->  175 ->  174 ->  173 ->  172 ->  171 ->  170 ->  169 ->  168 ->  167 ->  166 ->  165 ->  164 ->  163 ->  162 ->  161 ->  160 ->  159 ->  158 ->  157 ->  156 ->  155 ->  154 ->  153 ->  152 ->  151 ->  150 ->  149 ->  148 ->  147 ->  146 ->  145 ->  144 ->  143 ->  142 ->  141 ->  140 ->  139 ->  138 ->  137 ->  136 ->  135 ->  134 ->  133 ->  132 ->  131 ->  130 ->  129 ->  128 ->  127 ->  126 ->  125 ->  124 ->  123 ->  122 ->  121 ->  120 ->  119 ->  118 ->  117 ->  116 ->  115 ->  114 ->  113 ->  1

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2553/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


Objective: 0
Route for vehicle 0:
 157 -> 157
Distance of the route: 0.0m

Route for vehicle 1:
 157 -> 157
Distance of the route: 0.0m

Route for vehicle 2:
 157 ->  156 ->  155 ->  154 ->  153 ->  152 ->  151 ->  150 ->  149 ->  148 ->  147 ->  146 ->  145 ->  144 ->  143 ->  142 ->  141 ->  140 ->  139 ->  138 ->  137 ->  136 ->  135 ->  134 ->  133 ->  132 ->  131 ->  130 ->  129 ->  128 ->  127 ->  126 ->  125 ->  124 ->  123 ->  122 ->  121 ->  120 ->  119 ->  118 ->  117 ->  116 ->  115 ->  114 ->  113 ->  112 ->  111 ->  110 ->  109 ->  108 ->  107 ->  106 ->  105 ->  104 ->  103 ->  102 ->  101 ->  100 ->  99 ->  98 ->  97 ->  96 ->  95 ->  94 ->  93 ->  92 ->  91 ->  90 ->  89 ->  88 ->  87 ->  86 ->  85 ->  84 ->  83 ->  82 ->  81 ->  80 ->  79 ->  78 ->  77 ->  76 ->  75 ->  74 ->  73 ->  72 ->  71 ->  70 ->  69 ->  68 ->  67 ->  66 ->  65 ->  64 ->  63 ->  62 ->  61 ->  60 ->  59 ->  58 ->  57 ->  56 ->  55 ->  54 ->  53 ->  52 ->  51 ->  50 ->  49 ->  48 ->  47 ->  46 -> 

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2553/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


...

Objective: 0
Route for vehicle 0:
 164 -> 164
Distance of the route: 0.0m

Route for vehicle 1:
 164 -> 164
Distance of the route: 0.0m

Route for vehicle 2:
 164 ->  163 ->  162 ->  161 ->  160 ->  159 ->  158 ->  157 ->  156 ->  155 ->  154 ->  153 ->  152 ->  151 ->  150 ->  149 ->  148 ->  147 ->  146 ->  145 ->  144 ->  143 ->  142 ->  141 ->  140 ->  139 ->  138 ->  137 ->  136 ->  135 ->  134 ->  133 ->  132 ->  131 ->  130 ->  129 ->  128 ->  127 ->  126 ->  125 ->  124 ->  123 ->  122 ->  121 ->  120 ->  119 ->  118 ->  117 ->  116 ->  115 ->  114 ->  113 ->  112 ->  111 ->  110 ->  109 ->  108 ->  107 ->  106 ->  105 ->  104 ->  103 ->  102 ->  101 ->  100 ->  99 ->  98 ->  97 ->  96 ->  95 ->  94 ->  93 ->  92 ->  91 ->  90 ->  89 ->  88 ->  87 ->  86 ->  85 ->  84 ->  83 ->  82 ->  81 ->  80 ->  79 ->  78 ->  77 ->  76 ->  75 ->  74 ->  73 ->  72 ->  71 ->  70 ->  69 ->  68 ->  67 ->  66 ->  65 ->  64 ->  63 ->  62 ->  61 ->  60 ->  59 ->  58 ->  57 ->  56 ->  55 ->  54 -> 

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2553/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


Objective: 0
Route for vehicle 0:
 341 -> 341
Distance of the route: 0.0m

Route for vehicle 1:
 341 -> 341
Distance of the route: 0.0m

Route for vehicle 2:
 341 ->  340 ->  339 ->  338 ->  337 ->  336 ->  335 ->  334 ->  333 ->  332 ->  331 ->  330 ->  329 ->  328 ->  327 ->  326 ->  325 ->  324 ->  323 ->  322 ->  321 ->  320 ->  319 ->  318 ->  317 ->  316 ->  315 ->  314 ->  313 ->  312 ->  311 ->  310 ->  309 ->  308 ->  307 ->  306 ->  305 ->  304 ->  303 ->  302 ->  301 ->  300 ->  299 ->  298 ->  297 ->  296 ->  295 ->  294 ->  293 ->  292 ->  291 ->  290 ->  289 ->  288 ->  287 ->  286 ->  285 ->  284 ->  283 ->  282 ->  281 ->  280 ->  279 ->  278 ->  277 ->  276 ->  275 ->  274 ->  273 ->  272 ->  271 ->  270 ->  269 ->  268 ->  267 ->  266 ->  265 ->  264 ->  263 ->  262 ->  261 ->  260 ->  259 ->  258 ->  257 ->  256 ->  255 ->  254 ->  253 ->  252 ->  251 ->  250 ->  249 ->  248 ->  247 ->  246 ->  245 ->  244 ->  243 ->  242 ->  241 ->  240 ->  239 ->  238 ->  237 ->  2

/var/folders/tp/dcr77srx31sfjt_0ldrqb0hh0000gn/T/ipykernel_2553/3051611209.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  newDF.append(arrive,ignore_index=True)


Objective: 0
Route for vehicle 0:
 99 -> 99
Distance of the route: 0.0m

Route for vehicle 1:
 99 -> 99
Distance of the route: 0.0m

Route for vehicle 2:
 99 ->  98 ->  97 ->  96 ->  95 ->  94 ->  93 ->  92 ->  91 ->  90 ->  89 ->  88 ->  87 ->  86 ->  85 ->  84 ->  83 ->  82 ->  81 ->  80 ->  79 ->  78 ->  77 ->  76 ->  75 ->  74 ->  73 ->  72 ->  71 ->  70 ->  69 ->  68 ->  67 ->  66 ->  65 ->  64 ->  63 ->  62 ->  61 ->  60 ->  59 ->  58 ->  57 ->  56 ->  55 ->  54 ->  53 ->  52 ->  51 ->  50 ->  49 ->  48 ->  47 ->  46 ->  45 ->  44 ->  43 ->  42 ->  41 ->  40 ->  39 ->  38 ->  37 ->  36 ->  35 ->  34 ->  33 ->  32 ->  31 ->  30 ->  29 ->  28 ->  27 ->  26 ->  25 ->  24 ->  23 ->  22 ->  21 ->  20 ->  19 ->  18 ->  17 ->  16 ->  15 ->  14 ->  13 ->  12 ->  11 ->  10 ->  9 ->  8 ->  7 ->  6 ->  5 ->  4 ->  3 ->  2 ->  1 ->  0 -> 99
Distance of the route: 0.0m

Maximum of the route distances: 0.0m
[[99, 99], [99, 99], [99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 8

In [11]:
answerDF = pd.DataFrame(answer)